In [7]:

# pragma pylint: disable=missing-docstring, invalid-name, pointless-string-statement
# flake8: noqa: F401
# isort: skip_file
# --- Do not remove these libs ---
import sys
sys.path.append("/home/andy/CryptoTradingPlatform/freqtrade")

import numpy as np  # noqa
import pandas as pd  # noqa
from pandas import DataFrame
from typing import Optional, Union
from freqtrade.strategy import (BooleanParameter, CategoricalParameter, DecimalParameter,
                                IStrategy, IntParameter)
from typing import Dict, List, Optional, Tuple, Union
from freqtrade.enums import (CandleType,SignalTagType,SignalType, SignalDirection, SignalTagType, TRADING_MODES, TradingMode)
from freqtrade.constants import Config
from datetime import datetime, timezone


# --------------------------------
# Add your lib to import here
import talib.abstract as ta
import freqtrade.vendor.qtpylib.indicators as qtpylib
import logging
logger = logging.getLogger(__name__)
from freqtrade.data.cus_dataprovider import CustomDataProvider
from freqtrade.exchange.exchange_utils import timeframe_to_seconds

In [6]:
# This class is a sample. Feel free to customize it.

# Customized parameters
MACD_FAST = 26
MACD_SLOW= 12
MACD_SIGNAL= 9
SHORT_MULTIPLES = 1
MEDIUM_MULTIPLES = 3
LONG_MULTIPLES = 8
TRADE_TIMEFRAME_IN_MS = 4 * 60 * 60 * 1000
MARKET_CYCLE_SPEED = ["FAST","MEDIUM","SLOW"]

class ArbitrageAltBTC(IStrategy):
    """
    This is a sample strategy to inspire you.
    More information in https://www.freqtrade.io/en/latest/strategy-customization/

    You can:
        :return: a Dataframe with all mandatory indicators for the strategies
    - Rename the class name (Do not forget to update class_name)
    - Add any methods you want to build your strategy
    - Add any lib you need to build your strategy

    You must keep:
    - the lib in the section "Do not remove these libs"
    - the methods: populate_indicators, populate_entry_trend, populate_exit_trend
    You should keep:
    - timeframe, minimal_roi, stoploss, trailing_*
    """
    # Strategy interface version - allow new iterations of the strategy interface.
    # Check the documentation or the Sample strategy to get the latest version.
    INTERFACE_VERSION = 3

    
    dp: CustomDataProvider
    
    # Can this strategy go short?
    can_short: bool = True

    # Minimal ROI designed for the strategy.
    # This attribute will be overridden if the config file contains "minimal_roi".
    minimal_roi = {
        "0": 0.9
    }

    # Optimal stoploss designed for the strategy.
    # This attribute will be overridden if the config file contains "stoploss".
    stoploss = -0.5

    # Trailing stoploss
    trailing_stop = False
    # trailing_only_offset_is_reached = False
    # trailing_stop_positive = 0.01
    # trailing_stop_positive_offset = 0.0  # Disabled / not configured

    # Optimal timeframe for the strategy.
    timeframe = "4h"

    # Run "populate_indicators()" only for new candle.
    process_only_new_candles = True

    # These values can be overridden in the config.
    use_exit_signal = False
    exit_profit_only = False
    ignore_roi_if_entry_signal = False
    startup_candle_count: int = 365

    # Optional order type mapping.
    order_types = {
        'entry': 'market',
        'exit': 'market',
        'stoploss': 'market',
        'stoploss_on_exchange': False
    }

    # Optional order time in force.
    order_time_in_force = {
        'entry': 'GTC',
        'exit': 'GTC'
    }
    
    selected_symbols = []

    plot_config = {
        'main_plot': {
            'tema': {},
            'sar': {'color': 'white'},
        },
        'subplots': {
            "MACD": {
                'macd': {'color': 'blue'},
                'macdsignal': {'color': 'orange'},
            },
            "RSI": {
                'rsi': {'color': 'red'},
            }
        }
    }

    def __init__(self,config:Config,params:Dict):
        super().__init__(config)
        self._last_candle_seen = None
        self.market_cycle= params["market_cycle"]
        self.altcoin_cycle = params["altcoin_cycle"]
        self.n_pairs = params["n_pairs"]
    
    def get_selected_symbols(self):
        selected_symbols = []
        return selected_symbols
    
    def get_exit_signal(
        self,
        pair: str,
        timeframe: str,
        dataframe: DataFrame,
        is_short: Optional[bool] = None
        ) -> Tuple[bool, bool, Optional[str]]:
            """
            Calculates current exit signal based based on the dataframe
            columns of the dataframe.
            Used by Bot to get the signal to exit.
            depending on is_short, looks at "short" or "long" columns.
            :param pair: pair in format ANT/BTC
            :param timeframe: timeframe to use
            :param dataframe: Analyzed dataframe to get signal from.
            :param is_short: Indicating existing trade direction.
            :return: (enter, exit) A bool-tuple with enter / exit values.
            """
            latest, latest_date = self.get_latest_candle(pair, timeframe, dataframe)
            
            if latest is None:
                return False, False, None

            if is_short:
                enter_ = latest.get(f"{pair}_{SignalType.ENTER_SHORT.value}", 0) == 1
                exit_ = latest.get(f"{pair}_{SignalType.EXIT_SHORT.value}", 0) == 1

            else:
                enter_ = latest.get(f"{pair}_{SignalType.ENTER_LONG.value}",0) == 1
                exit_ = latest.get(f"{pair}_{SignalType.EXIT_LONG.value}", 0) == 1
            
            exit_tag = latest.get(f"{pair}_{SignalTagType.EXIT_TAG.value}", None)
            # Tags can be None, which does not resolve to False.
            exit_tag = exit_tag if isinstance(exit_tag, str) else None

            logger.debug(f"exit-trigger: {latest['date']} (pair={pair}) "
                        f"enter={enter_} exit={exit_}")

            return enter_, exit_, exit_tag

    def get_entry_signal(
        self,
        pair: str,
        timeframe: str,
        dataframe: DataFrame,
    ) -> Tuple[Optional[SignalDirection], Optional[str]]:
        """
        Calculates current entry signal based based on the dataframe signals
        columns of the dataframe.
        Used by Bot to get the signal to enter trades.
        :param pair: pair in format ANT/BTC
        :param timeframe: timeframe to use
        :param dataframe: Analyzed dataframe to get signal from.
        :return: (SignalDirection, entry_tag)
        """
        latest, latest_date = self.get_latest_candle(pair, timeframe, dataframe)
        if latest is None or latest_date is None:
            return None, None

        enter_long = latest.get(f"{pair}_{SignalType.ENTER_LONG.value}",0) == 1
        enter_short = latest.get(f"{pair}_{SignalType.ENTER_SHORT.value}",0) == 1
        exit_long = latest.get(f"{pair}_{SignalType.EXIT_LONG.value}",0) == 1
        exit_short = latest.get(f"{pair}_{SignalType.EXIT_SHORT.value}",0) == 1
        
        enter_signal: Optional[SignalDirection] = None
        enter_tag_value: Optional[str] = None
        
        if enter_long == 1 and not any([exit_long, enter_short]):
            enter_signal = SignalDirection.LONG
            enter_tag_value = latest.get(f"{pair}_{SignalTagType.ENTER_TAG.value}",0)
        
        if (self.config.get('trading_mode', TradingMode.SPOT) != TradingMode.SPOT
                and self.can_short
                and enter_short == 1 and not any([exit_short, enter_long])):
            enter_signal = SignalDirection.SHORT
            enter_tag_value = latest.get(SignalTagType.ENTER_TAG.value, None)

        enter_tag_value = enter_tag_value if isinstance(enter_tag_value, str) else None

        timeframe_seconds = timeframe_to_seconds(timeframe)

        if self.ignore_expired_candle(
            latest_date=latest_date.datetime,
            current_time=datetime.now(timezone.utc),
            timeframe_seconds=timeframe_seconds,
            enter=bool(enter_signal)
        ):
            return None, enter_tag_value

        logger.debug(f"entry trigger: {latest['date']} (pair={pair}) "
                     f"enter={enter_long} enter_tag_value={enter_tag_value}")
        return enter_signal, enter_tag_value
    
    def analyze(self,pairs:List[str]):
        """Overide the function to calculate all indicators and
        enter, exit signal at once.

        Args:
            pairs (List[str]): _description_
        """
        
        dfs:List[pd.DataFrame] = []
        for pair in pairs:
            df = self.dp.ohlcv(
                pair, self.timeframe, 
                candle_type=self.config.get('candle_type_def',CandleType.SPOT)
            )
            df.add_prefix(pair)
            dfs.append(df)
  
        dataframe = pd.concat([dfs],axis=1,keys=["date"])        
        dataframe = self._analyze_tickers_internal(dataframe,pairs)
    
    
    def _advise_indicators_for_all_pairs(self,dataframe,pairs):
        dataframe = self._preprocessing(dataframe,pairs)
        dataframe = self._calculate_macd_diff_with_btc(dataframe,pairs)
        dataframe = self._calculate_macd_diff_mean(dataframe,pairs)
        return dataframe
    
    def _analyze_tickers_internal(self,dataframe:DataFrame,pairs):
        
        new_candle = self._last_candle_seen != dataframe.iloc[-1]['date']   
        if not self.process_only_new_candles or new_candle:
            
            #Analyzing, adding MACD, MACD diff, MACD mean indicator
            dataframe = self._advise_indicators_for_all_pairs(dataframe,pairs)
            #Adding entry signal
            dataframe = self._advise_entry_signal_for_all_pairs(dataframe,pairs)
            #Adding entry signal
            dataframe = self._advise_exit_singnal_for_all_pairs(dataframe,pairs)
            
            
            candle_type = self.config.get('candle_type_def', CandleType.SPOT)
            # self.dp._emit_df((pair, self.timeframe, candle_type), dataframe, new_candle)
            self.dp._set_cached_super_df(self.timeframe, dataframe, candle_type=candle_type)
            self._last_candle_seen = dataframe.iloc[-1]['date']
                   
        else:
            logger.debug("Skipping TA Analysis for already analyzed candle")
            dataframe = self._remove_all_entry_exit_signals(dataframe,pairs)
        
        logger.debug("Loop analysis launched")
        return dataframe
    
    def arbitrate_both_sides(self,row,pairs,market_cycle,altcoin_cycle,n_pairs):
        is_bull = False
        long_pairs = short_pairs = []
        
        if row[f"BTC:USDT/USDT_{market_cycle}_MACD"] > 0:
            is_bull = True
        
        long_dict = dict()  
        short_dict = dict()
        inv_long_dict = dict()
        inv_short_dict = dict()
        
        for pair in pairs:
            if (is_bull):
                diff = float(row[f"{pair}_DIFF_{altcoin_cycle}_BULL"])
                diff_mean = float(row[f"{pair}_DIFF_{altcoin_cycle}_BULL_MEAN"])
            else:
                diff = float(row[f"{pair}_DIFF_{altcoin_cycle}_BEAR"])
                diff_mean = float(row[f"{pair}_DIFF_{altcoin_cycle}_BEAR_MEAN"])
            
            if (diff_mean > 0) & (diff > 0):
                long_dict[pair] = diff_mean
                inv_long_dict[diff_mean] = pair
            elif (diff_mean < 0) & (diff < 0):
                short_dict[pair] = diff_mean
                inv_short_dict[diff_mean] = pair
                
        sorted_inv_long_dict = sorted(inv_long_dict,reverse=True)
        long_pairs = [inv_long_dict[sorted_inv_long_dict[i]] for i in range(min(len(long_dict),n_pairs))]
            
        sorted_inv_short_dict = sorted(inv_short_dict,reverse=False)
        short_pairs = [inv_short_dict[sorted_inv_short_dict[i]] for i in range(min(len(short_dict),n_pairs))]
        
        long_dict["BTCUSDT"] = 0
        short_dict["BTCUSDT"] = 0
        
        if long_pairs and not short_pairs:
            short_pairs += ["BTCUSDT"]
        elif short_pairs and not long_pairs:
            long_pairs += ["BTCUSDT"]
        
        return long_pairs, short_pairs
        
    def _advise_entry_signal_for_all_pairs(self,dataframe,pairs):
        latest=dataframe.iloc[-1]
        for pair in pairs:
            latest[f"{pair}_{SignalType.ENTER_LONG.value}"] = 0
            latest[f"{pair}_{SignalType.ENTER_SHORT.value}"] = 0
            latest[f"{pair}_{SignalTagType.ENTER_TAG.value}"] = 0
        
        long_pairs, short_pairs = self.arbitrate_both_sides(
            row = latest,
            pairs= pairs,
            market_cycle=self.market_cycle,
            altcoin_cycle=self.altcoin_cycle,
            n_pairs=self.n_pairs)
        
        for pair in long_pairs:
            latest[f"{pair}_{SignalType.ENTER_LONG.value}"] = 1
        for pair in short_pairs:
            latest[f"{pair}_{SignalType.ENTER_SHORT.value}"] = 1
        
        return dataframe
    
    def _advise_exit_singnal_for_all_pairs(self,dataframe,pairs):
        latest = dataframe.iloc[-1]
        for pair in pairs:
            latest[f"{pair}_{SignalType.EXIT_LONG.value}"] = 0
            latest[f"{pair}_{SignalType.EXIT_SHORT.value}"] = 0
            latest[f"{pair}_{SignalTagType.EXIT_TAG.value}"] = 0
        
        return dataframe
    
    def _remove_all_entry_exit_signals(self,dataframe,pairs):
        """
        Remove Entry and Exit signals from a DataFrame
        :param dataframe: The DataFrame to remove signals from
        """
        for pair in pairs:
            dataframe[f"{pair}_{SignalType.ENTER_LONG.value}"] = 0
            dataframe[f"{pair}_{SignalType.EXIT_LONG.value}"] = 0
            dataframe[f"{pair}_{SignalType.ENTER_SHORT.value}"] = 0
            dataframe[f"{pair}_{SignalType.EXIT_SHORT.value}"] = 0
            dataframe[f"{pair}_{SignalTagType.ENTER_TAG.value}"] = 0
            dataframe[f"{pair}_{SignalTagType.EXIT_TAG.value}"] = 0
           
        return dataframe             
    def informative_pairs(self):
        """
        Define additional, informative pair/interval combinations to be cached from the exchange.
        These pair/interval combinations are non-tradeable, unless they are part
        of the whitelist as well.
        For more information, please consult the documentation
        :return: List of tuples in the format (pair, interval)
            Sample: return [("ETH/USDT", "5m"),
                            ("BTC/USDT", "15m"),
                            ]
        """
        return []

    def _preprocessing(self, df:pd.DataFrame,pairs):
        print("...preprocessing, calculate MACD ...")
        df = df.copy()
        
        for pair in pairs:
            # log_return_series = np.log(df[f"{symbol}_close"].pct_change()+1)
            # log_return_series.name = f"{symbol}_LOG_RETURN"
            
            _,_,macd_medium_series = ta.MACD(df[f"{pair}_close"],fastperiod = MACD_FAST, \
                slowperiod=MACD_SLOW,signalperiod=MACD_SIGNAL)
            macd_medium_series.name = df[f"{pair}_MEDIUM_MACD"]
            
            df[f"{pair}_SLOW_MACD"] = df[f"{pair}_SLOW_MACD"]/df["close"]
            _,_,macd_slow_series = ta.MACD(df[f"{pair}_close"],fastperiod=MACD_FAST, \
                slowperiod=MACD_SLOW,signalperiod=MACD_SIGNAL)
            macd_slow_series.name = df[f"{pair}_SLOW_MACD"]
            df = pd.concat([df,macd_medium_series,macd_slow_series],axis=1)
        
        df.sort_index(ascending=True,inplace=True)
        df.fillna(method="ffill", inplace = True)
        df.dropna(inplace=True)
        return df
    
    def _calculate_macd_diff_with_btc(self, df:pd.DataFrame,pairs:list[str]):
        print("...calculating MACD diff between altcoins and BTC...")
        df = df.copy()
        #Calculating diff
        for symbol in pairs:
            for cycle in MARKET_CYCLE_SPEED:
                diff = df[f"{symbol}_{cycle}_MACD"] - df[f"BTCUSDT_{cycle}_MACD"]
                diff.name = f"{symbol}_DIFF_{cycle}"
                
                diff_bear = df[df["BTCUSDT_SLOW_MACD"]<0][f"{symbol}_{cycle}_MACD"] \
                    - df[df["BTCUSDT_SLOW_MACD"]<0][f"BTCUSDT_{cycle}_MACD"]
                diff_bear.name = f"{symbol}_DIFF_{cycle}_BEAR"
                
                diff_bull = df[df["BTCUSDT_SLOW_MACD"]>0][f"{symbol}_{cycle}_MACD"] \
                - df[df["BTCUSDT_SLOW_MACD"]>0][f"BTCUSDT_{cycle}_MACD"]
                diff_bull.name = f"{symbol}_DIFF_{cycle}_BULL"
                
                df = pd.concat([df,diff,diff_bear,diff_bull],axis=1)     
                
        return df 
    
    def _calculate_macd_diff_mean(self,merged_df,selected_symbols):
        #Calculating rolling diff mean
        df = merged_df.copy()
        print("...calculating MACD diff mean...")
        for symbol in selected_symbols:
            for cycle in MARKET_CYCLE_SPEED:
                diff_bear_mean = df[f"{symbol}_DIFF_{cycle}_BEAR"].expanding().mean() 
                diff_bear_mean.name = f"{symbol}_DIFF_{cycle}_BEAR_MEAN"
                diff_bull_mean = df[f"{symbol}_DIFF_{cycle}_BULL"].expanding().mean() 
                diff_bull_mean.name = f"{symbol}_DIFF_{cycle}_BULL_MEAN"
                df = pd.concat([df,diff_bear_mean,diff_bull_mean],axis=1)

        return df
    
    def populate_indicators(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        """
        Adds several different TA indicators to the given DataFrame

        Performance Note: For the best performance be frugal on the number of indicators
        you are using. Let uncomment only the indicator you are using in your strategies
        or your hyperopt configuration, otherwise you will waste your memory and CPU usage.
        :param dataframe: Dataframe with data from the exchange
        :param metadata: Additional information, like the currently traded pair
        :return: a Dataframe with all mandatory indicators for the strategies
        """
        
        return dataframe

    
    def populate_entry_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        """
        Based on TA indicators, populates the entry signal for the given dataframe
        :param dataframe: DataFrame
        :param metadata: Additional information, like the currently traded pair
        :return: DataFrame with entry columns populated
        """
        return dataframe

    def populate_exit_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        """
        Based on TA indicators, populates the exit signal for the given dataframe
        :param dataframe: DataFrame
        :param metadata: Additional information, like the currently traded pair
        :return: DataFrame with exit columns populated
        """
        return dataframe